# Archivo Historico de Localidades

En esta libreta se realizará una exploración inicial de los archivos y se unieran todos los registros que vienen divididos por estado en un solo CSV.

En General INEGI sigue las guias para compartir informacion internacionales que utilizan 'utf-8' como codificación estandar pero hay veces que los datos que ofrece tienen la codificación 'Windows-1252' (codigo 'cp1252') o 'Latin-1' (codigo 'latin-1'). Esto sucede generalmente cuando se manipulan y exportan los datos desde Excel o tal vez Access y es el caso de estos archivos. Los archivos unidos se exportaran en 'utf-8'.

## Librerias y configuraciones

In [1]:
# Librerias

import os
import shutil
from zipfile import ZipFile
import csv
import random

In [2]:
verbose = True  # Imprimir notas de ejecución

## Contenido de los zips

El archivo base es un zip llamado 'AHL.zip' dentro de la carpeta 'Datos' que tiene un tamaño comprimido de 78 megabytes.

In [3]:
dir_datos = 'Datos'                                      # Sub-directorio del zip
nom_archivo = 'AHL.zip'                                  # Nombre del zip
ruta_zip_general = os.path.join(dir_datos, nom_archivo)          # Ruta del archivo zip


if verbose:
    print('Contenido de carpeta datos...')
    print(os.listdir(dir_datos))
    print('Tamaño:')
    print(os.path.getsize(ruta_zip_general)/1_000_000, 'Mb')

Contenido de carpeta datos...
['AHL.zip']
Tamaño:
78.002994 Mb


Contiene 32 zips uno para cada estado de la republica) con 5 archivos cada uno:
* Habitantes.csv
* Maestro.csv
* Movimientos.csv
* Res_hist.csv
* Estructura_Archivos_AHL.pdf (descriptor de los archivos CSV, es el mismo para todos Estados.)

In [4]:
# AHL.zip
dir_extraccion = 'temp_ahl'  # Carpeta donde se van a descomprimir los archivos

try:
    os.mkdir(dir_extraccion)    

except FileExistsError:
    print(f'El directorio {dir_extraccion} ya existe...')

with ZipFile(ruta_zip_general, 'r') as zip_file:
    lista_zips = zip_file.namelist()

    # Seleccionar y extraer un archivo aleatorio
    zip_estatal = lista_zips[random.randint(0, len(lista_zips) - 1)]
    zip_file.extract(zip_estatal, dir_extraccion)


# Zip Estatal    
nom_zip_estatal = zip_estatal[0:5]          # Nombre del zip sin extension
dir_zip_estatal = os.path.join(dir_extraccion, nom_zip_estatal) # Directorio a crear
ruta_zip_estatal = os.path.join(dir_extraccion, zip_estatal) # Ruta del zip
try:
    os.mkdir(dir_zip_estatal)     
except FileExistsError:
    print(f'El directorio {dir_zip_estatal} ya existe...')


with ZipFile(ruta_zip_estatal) as info_estatal:
    lista_estatal = info_estatal.namelist()
    
    # Extraer todos los archivos
    info_estatal.extractall(dir_zip_estatal)
    
if verbose:
    print(f'Numero de archivos en el zip prinicipal: {len(lista_zips)}', end = '\n\n' )
    print('Contenido del zip:', end = '\n\n')
    print(lista_zips, end = '\n\n')
    print('Contenido de los zips contenidos en AHL.zip:', end = '\n\n')
    print(lista_estatal)

  

Numero de archivos en el zip prinicipal: 32

Contenido del zip:

['AHL01.zip', 'AHL02.zip', 'AHL03.zip', 'AHL04.zip', 'AHL05.zip', 'AHL06.zip', 'AHL07.zip', 'AHL08.zip', 'AHL09.zip', 'AHL10.zip', 'AHL11.zip', 'AHL12.zip', 'AHL13.zip', 'AHL14.zip', 'AHL15.zip', 'AHL16.zip', 'AHL17.zip', 'AHL18.zip', 'AHL19.zip', 'AHL20.zip', 'AHL21.zip', 'AHL22.zip', 'AHL23.zip', 'AHL24.zip', 'AHL25.zip', 'AHL26.zip', 'AHL27.zip', 'AHL28.zip', 'AHL29.zip', 'AHL30.zip', 'AHL31.zip', 'AHL32.zip']

Contenido de los zips contenidos en AHL.zip:

['AHL10Habitantes.csv', 'AHL10Maestro.csv', 'AHL10Movimientos.csv', 'AHL10Res_hist.csv', 'Estructura_Archivos_AHL.pdf']


In [5]:
# Obtener muestra de CSVs
muestra_csvs = {}
temas = ['Habitantes','Maestro','Movimientos','Res_hist']

for tema in temas:
    nom_csv = nom_zip_estatal + tema + '.csv'
    ruta_csv = os.path.join(dir_zip_estatal, nom_csv)
    
    try:
        with open(ruta_csv, 'r', encoding="cp1252") as row: # Se intento con utf-8 pero habia problemas
            reader = csv.reader(row)
            header = next(reader,'unix')
            data_sample = next(reader)
            muestra_csvs[tema] = {key:value for key,value in zip(header, data_sample)}
        
        
        
    except FileNotFoundError:
        print(f'El archivo {nom_csv} no existe en la carpeta {nom_zip_estatal}...')

    except UnicodeDecodeError:
        print("Intenta otra codificiación. En general los archivos en México",
        " estan codificados en utf-8 o cp1252.")

In [6]:
# Borrar carpeta temporal
shutil.rmtree(dir_extraccion)

En todas las tablas aparece la columna CVE_GEOEST. Habitantes y Movimientos comparten la Columna CLAVE. El resto de las columnas son unicas a las tablas que pertenecen. En algunas columnas existen valores nulos.

In [7]:
if verbose:
    for tema in temas[:3]:
        print(f'Muestra de Columna/Valor de CSV {tema}:')
        for item in muestra_csvs[tema].items(): print(item)
        print()
    
    print(f'Muestra de Columna/Valor de CSV {tema[3]}:')
    for key, value in muestra_csvs[temas[3]].items(): 
        reducido = value.replace('\n', '')[:50]
        print(f'(\'{key}\', \'{reducido}\'...)')

Muestra de Columna/Valor de CSV Habitantes:
('CLAVE', '112305')
('CVE_GEOEST', '100010001')
('EVE_CENSAL', '1980')
('INDICE_HAB', '15474324')
('FUENTE', 'Censo')
('TOT_HAB', '8,384')
('TOT_HOM', '0')
('TOT_MUJ', '0')

Muestra de Columna/Valor de CSV Maestro:
('CVE_GEOEST', '100020032')
('LATITUD', '')
('LONGITUD', '')
('ALTITUD', '')
('CARTA_TOPO', '')
('TIPO', 'R')
('NOMBRE_EDO', 'Durango')
('NOMBRE_MUN', 'Canelas')

Muestra de Columna/Valor de CSV Movimientos:
('CLAVE', '5496110')
('CVE_GEOEST', '100170070')
('INDICE', '22')
('NOM_LOC', 'Villa Orestes Pereyra (Rosario)')
('NOM_MUN', 'Ocampo')
('CAT_POLI', 'Ranchería')
('CAT_ADMIVA', '')
('ORI_MODIF', 'Censo de 2020.')

Muestra de Columna/Valor de CSV i:
('CVE_GEOEST', '100010001'...)
('DATOS', 'Canatlán-   La palabra Canatlán viene del náhuatl '...)
('DATOS_BIBLIOGRAFIA', 'Bibliografía:(1)  http://www.inafed.gob.mx/work/en'...)


## Descomprimir csvs por tema

 En esta sección se descomprimen los csvs en una carpeta llamada CSVs dentro de otra subcarpeta para cada tema.

### Funciones

In [8]:
def extraer_archivo(ruta_zip, dir_extraccion, archivo_extraer, verbose = False):
    """
    Función para extraer un archivo dentro de un zip a una ruta determinada.
    predeterminada.
    
    :param ruta_zip: Ruta donde se encuentra zip en el cual se encuentra 
        el archivo.
    :param dir_extraccion: Directorio donde se va a extraer el archivo.
    :param archivo_extraer: Nombre del archivo a extraer.
    :param verbose: Imprimir comentarios en procedimiento.
    :return: No regresa nada, solo extrae archivos a pc.

    """
    if verbose:
        if not os.path.exists(dir_extraccion):
            os.makedirs(dir_extraccion)
            print(f'Creando directorio {dir_extraccion}')

    ruta_archivo_extraer = os.path.join(dir_extraccion, archivo_extraer)


    if verbose:
        if os.path.exists(ruta_archivo_extraer):
            print(f'El archivo {ruta_archivo_extraer} ya existe, sobreescribiendo...')
        
        else:
            print(f'Descomprimiendo archivo {ruta_archivo_extraer}')


    with ZipFile(ruta_zip, 'r') as zip_file:
        zip_file.extract(archivo_extraer, dir_extraccion)


In [9]:


def extraer_tema(dir_extraccion, tema,  dir_archivos, zipfile_list, verbose = False):
    """
    Función para extraer un archivo dentro de un zip a una ruta determinada.
    predeterminada.
    
    :param dir_extraccion: Directorio donde se van a extraer los archivos.
    :param tema: Tema que se va a extraer, pueden ser cuatro:
        Habitantes, Movimientos, Maestro, y Res_hist
    :param dir_archivos: Directorio donde se encuentran los zips de los cuales
        se va a extraer un tema.
    :param zipfile_list: Lista de archivos zip donde se encuentran los csvs
        por tema.
    :param verbose: Imprimir comentarios en procedimiento.
    
    :return: No regresa nada, solo extrae archivos a pc de un tema en un
        directorio determinado.
    """
    dir_extraccion_tema = os.path.join(dir_extraccion, tema)
    print(f'Descomprimiendo csvs de {tema} en {dir_extraccion_tema}', end='... ')
    for zip_file in zipfile_list:
        archivo_tema = zip_file[:5] + tema +'.csv'
        ruta_zip_estatal = os.path.join(dir_archivos, zip_file)
        extraer_archivo(ruta_zip_estatal, dir_extraccion_tema, archivo_tema, verbose = False)
    print(f'Extraidos.')


### Implementación csvs por tema

In [10]:
ruta_zip_general = 'Datos\AHL.zip'     # Variable ya definida creada anteriormente
dir_extraccion_estatales = 'temp_zips_estatales' # Directorio temporal donde se extraen los zips estatales
with ZipFile(ruta_zip_general, 'r') as zip_file:
    zip_file.extractall(dir_extraccion_estatales)

In [11]:
# Los temas dentro de los zips son los siguientes: ['Habitantes', 'Maestro', 'Movimientos', 'Res_hist']
dir_extraccion = 'CSVs'    # Carpeta donde se van a extraer los temas

extraer_tema(dir_extraccion, 'Habitantes', dir_extraccion_estatales, lista_zips, verbose)
extraer_tema(dir_extraccion, 'Maestro', dir_extraccion_estatales, lista_zips, verbose)
extraer_tema(dir_extraccion, 'Movimientos', dir_extraccion_estatales, lista_zips, verbose)
extraer_tema(dir_extraccion, 'Res_hist', dir_extraccion_estatales, lista_zips, verbose)

Descomprimiendo csvs de Habitantes en CSVs\Habitantes... Extraidos.
Descomprimiendo csvs de Maestro en CSVs\Maestro... Extraidos.
Descomprimiendo csvs de Movimientos en CSVs\Movimientos... Extraidos.
Descomprimiendo csvs de Res_hist en CSVs\Res_hist... Extraidos.


## Unir csvs por tema

Codigo para unir todos los csvs estatales en uno  solo.

### Funciones

In [12]:
def unir_csvs(unido, a_unir, con_header = False, verbose = False):
    """
    Función que lee un archivo csv y lo escribe en otro.
    
    :param unido: Archivo donde se va a anexar el csv.
    :param a_unir: Archivo que se va a anexar al csv unido.
    :param con_header: Nombre del csv que se va a extraer puede tener los valores de
    :param verbose: True para imprimir información de lo que esta haciedo.

    :return: No regresa nada, solo une csvs.

    """

    with open(a_unir, 'r', encoding="cp1252", errors='replace') as csv_a_unir:
        doc_reader = csv.reader(csv_a_unir)

        if con_header == True:
            with open(unido, 'w', encoding='utf-8', errors='replace') as unido:
                doc_writer = csv.writer(unido, lineterminator='\n')
                header = next(doc_reader)
                doc_writer.writerow(header)

                if verbose:
                    print(f'Header agregado: {header}')

                doc_writer.writerows(doc_reader)

                if verbose:
                    print(f'Filas de{a_unir} agregadas a {unido}.')

        else:
            header = next(doc_reader)
            with open(unido, 'a', encoding='utf-8') as unido:
                doc_writer = csv.writer(unido, lineterminator='\n')
                doc_writer.writerows(doc_reader)
                if verbose:
                    print(f'Filas de{a_unir} agregadas a {unido}.')

In [13]:
def unir_tema(ruta_csv, dir_carpeta, tema, verbose = False):
    """
    Función para extraer un archivo dentro de un zip a una ruta determinada.
    predeterminada.

    :param ruta_csv: Ruta del archivo donde se van a unir los csvs.
    :param dir_carpeta: Directorio donde se encuentra la carpeta.
    :param tema: Nombre con la carpeta con los csvs del tema.

    :return: No regresa nada, solo extrae archivos a pc de un tema en un
        directorio determinado.
    """
    if os.path.exists(ruta_csv):
        print(f'El archivo {ruta_csv} ya existe... borrandolo')
        os.remove(ruta_csv)

    con_header = True
    dir_tema = os.path.join(dir_carpeta, tema)
    csvs_a_unir = os.listdir(dir_tema)

    if verbose:
        print(f'Uniendo archivos de la carpeta {tema}', end = '... ')
        
    for csv in csvs_a_unir:
        ruta_csv_a_unir = os.path.join(dir_tema, csv)
        #print(ruta_csv, ruta_csv_a_unir, con_header, verbose) 
  
        unir_csvs(ruta_csv, ruta_csv_a_unir, con_header, verbose=False)

        con_header = False # Para solo copiar los encabezados del primer csv.

    if verbose:
        print(f'Unidos en {ruta_csv}')




### Implementación unir csvs por tema

In [14]:
# Unir los temas en carpeta csv.
# Existen errores de codificacion en los archivos 25 y 32 
# que se reemplazaron por ? con el parametro errors = 'replace'
# en la función unir_csvs.

unir_tema('CSVs\habitantes.csv', 'CSVs', 'Habitantes', verbose)
unir_tema('CSVs\maestro.csv', 'CSVs', 'Maestro', verbose)
unir_tema('CSVs\movimientos.csv', 'CSVs', 'Movimientos', verbose)
unir_tema(r'CSVs\res_hist.csv', 'CSVs', 'Res_hist', verbose) # tiene r antes del string para que
                                                             # /r se interprete bien


El archivo CSVs\habitantes.csv ya existe... borrandolo
Uniendo archivos de la carpeta Habitantes... Unidos en CSVs\habitantes.csv
El archivo CSVs\maestro.csv ya existe... borrandolo
Uniendo archivos de la carpeta Maestro... Unidos en CSVs\maestro.csv
El archivo CSVs\movimientos.csv ya existe... borrandolo
Uniendo archivos de la carpeta Movimientos... Unidos en CSVs\movimientos.csv
El archivo CSVs\res_hist.csv ya existe... borrandolo
Uniendo archivos de la carpeta Res_hist... Unidos en CSVs\res_hist.csv


In [15]:

for tema in temas:
    csv_general = tema.lower() + '.csv'
    ruta = os.path.join('CSVs', csv_general)
    print(f'Tamaño de {csv_general}:', end = ' ')
    print(os.stat(ruta).st_size / 1000000, 'megabytes')
    with open(ruta, 'r') as csv_tema:
        num_lineas = len(csv_tema.readlines()) - 1
        print(f'Numero de registros: {num_lineas}')

Tamaño de habitantes.csv: 98.951205 megabytes
Numero de registros: 2138702
Tamaño de maestro.csv: 33.13731 megabytes
Numero de registros: 409446
Tamaño de movimientos.csv: 221.355535 megabytes
Numero de registros: 3565344
Tamaño de res_hist.csv: 13.379794 megabytes
Numero de registros: 198847


In [16]:
# Borrar carpeta con los zips estatales
shutil.rmtree(dir_extraccion_estatales)

# Borrar carpeta de csvs estatales
shutil.rmtree('CSVs/Habitantes/')
shutil.rmtree('CSVs/Maestro/')
shutil.rmtree('CSVs/Movimientos/')
shutil.rmtree('CSVs/Res_hist/')